Faltten StructType, ArrayType

In [ ]:
from pyspark.sql.functions import col,explode_outer
from pyspark.sql.types import StructType,ArrayType


def flatten_struct(schema):
  
    def wrapper(schema,prefix = ""):
        flatten_cols = []        
        for field in schema.fields:
            path = f'{prefix}.{field.name}' if prefix  else f'{field.name}'
            if isinstance(field.dataType,StructType):
                  flatten_cols+=wrapper(field.dataType,path)
            else:
                  flatten_cols.append(path)
        return flatten_cols
    
    return [col(cols).alias(cols.replace('.','_')) for cols in wrapper(schema)]        


def flatten_array(df):
    
    # faltten structType
    df1 = df.select(flatten_struct(df.schema))
    
    # Get all Arraytype columns
    array_cols = [cols.name for cols in df1.schema.fields if isinstance(cols.dataType, ArrayType)]
    
    # initailize the lvl  
    lvl=1
    
    while len(array_cols)>0:
        
        # loop through each arraytype and explode them
        for c in array_cols:
            df1 = df1.select('*',explode_outer(f'`{c}`').alias(c.replace('.','_')+str(lvl))).drop(c)
        
        # rename column to replace .(Otherwise it will be treated as . oerator)
        df1 = df1.toDF(*(c.replace('.', '_') for c in df1.columns))
        
        # after explode faltten structType
        df1 = df1.select(flatten_struct(df1.schema))
        
        # check for arraytype in dataframe
        array_cols = [cols.name for cols in df1.schema.fields if isinstance(cols.dataType, ArrayType)]
        
        # increase level variable
        lvl+=1
    
    return df1

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import explode_outer, flatten


spark = SparkSession.builder.getOrCreate()
df = spark.read.format("com.databricks.spark.xml") \
    .option("rowTag", "catalog").load('/FileStore/temp1.xml')

display(df)

df = flatten_array(df)

display(df)

product
"List(Cardigan Sweater, cardigan.jpg, List(List(Men's, QWZ5671, 39.95, List(List(Medium, List(List(Red, red_cardigan.jpg), List(Burgundy, burgundy_cardigan.jpg))), List(Large, List(List(Red, red_cardigan.jpg), List(Burgundy, burgundy_cardigan.jpg))))), List(Women's, RRX9856, 42.5, List(List(Small, List(List(Red, red_cardigan.jpg), List(Navy, navy_cardigan.jpg), List(Burgundy, burgundy_cardigan.jpg))), List(Medium, List(List(Red, red_cardigan.jpg), List(Navy, navy_cardigan.jpg), List(Burgundy, burgundy_cardigan.jpg), List(Black, black_cardigan.jpg))), List(Large, List(List(Navy, navy_cardigan.jpg), List(Black, black_cardigan.jpg))), List(Extra Large, List(List(Burgundy, burgundy_cardigan.jpg), List(Black, black_cardigan.jpg)))))))"


product__description,product__product_image,product_catalog_item1__gender,product_catalog_item1_item_number,product_catalog_item1_price,product_catalog_item1_size2__description,product_catalog_item1_size2_color_swatch3__VALUE,product_catalog_item1_size2_color_swatch3__image
Cardigan Sweater,cardigan.jpg,Men's,QWZ5671,39.95,Medium,Red,red_cardigan.jpg
Cardigan Sweater,cardigan.jpg,Men's,QWZ5671,39.95,Medium,Burgundy,burgundy_cardigan.jpg
Cardigan Sweater,cardigan.jpg,Men's,QWZ5671,39.95,Large,Red,red_cardigan.jpg
Cardigan Sweater,cardigan.jpg,Men's,QWZ5671,39.95,Large,Burgundy,burgundy_cardigan.jpg
Cardigan Sweater,cardigan.jpg,Women's,RRX9856,42.5,Small,Red,red_cardigan.jpg
Cardigan Sweater,cardigan.jpg,Women's,RRX9856,42.5,Small,Navy,navy_cardigan.jpg
Cardigan Sweater,cardigan.jpg,Women's,RRX9856,42.5,Small,Burgundy,burgundy_cardigan.jpg
Cardigan Sweater,cardigan.jpg,Women's,RRX9856,42.5,Medium,Red,red_cardigan.jpg
Cardigan Sweater,cardigan.jpg,Women's,RRX9856,42.5,Medium,Navy,navy_cardigan.jpg
Cardigan Sweater,cardigan.jpg,Women's,RRX9856,42.5,Medium,Burgundy,burgundy_cardigan.jpg


Flatten MapType(Credit :sparkbyexmaple.com)

In [ ]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

dataDictionary = [
        ('James',{'hair':'black','eye':'brown'}),
        ('Michael',{'hair':'brown','eye':None}),
        ('Robert',{'hair':'red','eye':'black'}),
        ('Washington',{'hair':'grey','eye':'grey'}),
        ('Jefferson',{'hair':'brown','eye':''})
        ]

df = spark.createDataFrame(data=dataDictionary, schema = ['name','properties'])
df.printSchema()
# df.show(truncate=False)


from pyspark.sql.functions import explode,map_keys,col
keysDF = df.select(explode(map_keys(df.properties))).distinct()
keysList = keysDF.rdd.map(lambda x:x[0]).collect()
keyCols = list(map(lambda x: col("properties").getItem(x).alias(str(x)), keysList))
df.select(df.name, *keyCols).show()


root
 |-- name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-----+-----+
|      name|  eye| hair|
+----------+-----+-----+
|     James|brown|black|
|   Michael| null|brown|
|    Robert|black|  red|
|Washington| grey| grey|
| Jefferson|     |brown|
+----------+-----+-----+

